In [2]:
import laspy
from pathlib import Path
import numpy as np
from scipy import stats
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Dataset

In [6]:
data_path = Path('../data/SOAP/2019/output')
laz_files = [i for i in data_path.glob('*.laz')]

indexes = []
point_data_z = []
for laz_file in laz_files:
    plot_id = laz_file.stem
    indexes.append(plot_id)
    data = laspy.read(laz_file)
    point_data = np.stack([data.X, data.Y, data.Z], axis=0).transpose((1, 0)) * data.header.z_scale + data.header.z_offset
    mask = pd.Series(data.classification.array).isin([2,3,4,5])
    point_data = point_data[mask]
    d_z = point_data[:, 2]
    d = pd.Series(d_z).describe()
    d['skew'] = stats.skew(d_z)
    d['kurtosis'] = stats.kurtosis(d_z)
    point_data_z.append(d)
df = pd.DataFrame(point_data_z)
df['plotID'] = indexes

bio_df = pd.read_csv('../data/SOAP/2019/output/plot_level_pp_veg_structure_IND_IBA_IAGB_live.csv')
data_df = df.set_index('plotID').join(bio_df.set_index('plotID')).dropna()
data_df

4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095


,count,mean,std,min,25%,50%,75%,max,skew,kurtosis,stemNumberDensity,basalArea,biomass
plotID,,,,,,,,,,,,,
SOAP_004,1394.0,17.146499,13.228680,0.0,0.0,19.190,28.4050,47.01,-0.066510,-1.362612,0.1200,41.927460,197.363305
SOAP_019,1329.0,2.877065,7.677804,0.0,0.0,0.000,0.0000,30.80,2.467375,4.486459,0.0025,0.825159,0.232225
SOAP_017,1083.0,2.949381,5.531089,0.0,0.0,0.000,5.2400,25.90,2.011124,3.557178,0.2775,3.993141,69.180740
SOAP_020,1824.0,15.418755,9.719903,0.0,0.0,19.925,22.8925,27.76,-0.779021,-1.081771,0.6300,43.797336,154.895046
SOAP_007,3170.0,4.532562,5.588507,0.0,0.0,0.000,10.6000,24.24,0.640770,-1.176752,0.1500,14.393953,30.868115
SOAP_008,2299.0,10.715694,15.737930,0.0,0.0,0.000,22.5250,48.42,1.102373,-0.436621,0.0275,13.953934,7.564265
SOAP_012,1261.0,2.463695,5.677051,0.0,0.0,0.000,0.0000,23.89,2.169447,3.381671,0.0100,13.613660,5.350118
SOAP_013,2965.0,17.130550,15.566224,0.0,0.0,21.600,31.9700,46.67,0.032096,-1.661273,0.0175,54.945650,36.133307
SOAP_010,1217.0,1.199622,2.302337,0.0,0.0,0.000,1.8100,9.91,1.836022,2.314802,0.1850,4.330725,1.338252


# Train linear regression

In [7]:
X = data_df[['mean', 'std', 'skew', 'kurtosis']].values
y = data_df.biomass

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2)
model = LinearRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

train_score = model.score(X_train, y_train)

print(f'Mean absolute error: {mae:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')
actual_minus_predicted = sum((y_test - y_pred)**2)
actual_minus_actual_mean = sum((y_test - y_test.mean())**2)
r2 = 1 - actual_minus_predicted/actual_minus_actual_mean
print('R²:', r2)
print(f'Train score: {train_score:.2f}')

Mean absolute error: 38.16
Mean squared error: 4155.40
Root mean squared error: 64.46
R²: 0.37472864634502556
Train score: 0.75


In [8]:
pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

,Actual,Predicted
plotID,,
SOAP_010,1.338252,14.570965
SOAP_006,0.033420,-0.773292
SOAP_004,197.363305,69.579167
SOAP_009,33.705154,22.886328
